In [1]:
# STD MODULE
import os
import sys
import numpy as np
import cv2
import pandas as pd

# TORCH MODULE
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, utils

# PYTVISION MODULE
from pytvision.transforms import transforms as mtrans
from pytvision import visualization as view

# LOCAL MODULE
sys.path.append('../')
from torchlib.datasets.factory  import FactoryDataset 
from torchlib.datasets import Dataset, SecuencialSamplesDataset, TripletsDataset
from torchlib.neuralnet import TripletNeuralNet, TripletEnsambleNeuralNet
from misc import get_transforms_aug, get_transforms_det
from torchlib.ensamble import PoolNet


import matplotlib.pyplot

In [2]:
pathname = os.path.expanduser( '~/.datasets' )
name_dataset=FactoryDataset.ferp
project='../out/tripletruns'
cuda=False
seed=1
gpu=1
batch_size=30
workers=30
num_input_channels=3
parallel=False

print(pathname)
print(name_dataset)


/home/fernandez/.datasets
ferp


In [3]:
!ls ../out/tripletruns/

triplet_preactresembnet18_hinge_adam_ferp_emb32_001
triplet_resnetemb18_hinge_adam_ferp_emb32_001


In [5]:
cudnn.benchmark = True
dataprojects = [
    ['triplet_preactresembnet18_hinge_adam_ferp_emb32_001', 'model_best.pth.tar'],   
    ['triplet_preactresembnet18_hinge_adam_ferp_emb32_001', 'model_best.pth.tar'],
]

ensnet = TripletEnsambleNeuralNet( 
    no_cuda=cuda, 
    parallel=parallel, 
    seed=seed, 
    gpu=gpu, 
    )
ensnet.create(dataprojects, project)

    

Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


=> loading checkpoint '../out/tripletruns/triplet_preactresembnet18_hinge_adam_ferp_emb32_001/models/model_best.pth.tar'


Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


=> loaded checkpoint for preactresembnet18 arch!
=> loading checkpoint '../out/tripletruns/triplet_preactresembnet18_hinge_adam_ferp_emb32_001/models/model_best.pth.tar'
=> loaded checkpoint for preactresembnet18 arch!


In [6]:
# validation dataset
subset = FactoryDataset.validation
size_input = 64

data = Dataset(
    data=FactoryDataset.factory(
        pathname=pathname, 
        name=name_dataset, 
        subset=subset, 
        download=False 
    ),
    num_channels=num_input_channels,
    transform=get_transforms_det( size_input )
    )
dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=workers )


print(subset)
print(data.classes)
print( len(data) )
print(len(dataloader))


val
[0 1 2 3 4 5 6 7]
3191
107


In [7]:
# representation data
Z, Y = ensnet.representation( dataloader )
print('DONE!!')

100%|██████████| 107/107 [00:06<00:00, 17.81it/s]

DONE!!


In [7]:
print(Z.shape)
print(Y.shape)

(3191, 32, 3)
(3191,)


In [12]:
image = data[0]['image']
image = image.unsqueeze(dim=0)
print(image.shape)

zs = ensnet( image )
print(zs.shape)


torch.Size([1, 3, 64, 64])
torch.Size([1, 32, 2])
